In [32]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
import sqlalchemy as sa

import database as db

from images import get_list_of_images_invalid, separate_and_copy_images
from models import County, DataTrustedIdentifier

## Database connection

In [34]:
engine, session = db.connect()
engine.echo = False

2023-03-27 20:45:24,151 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-27 20:45:24,152 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 20:45:24,153 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-27 20:45:24,153 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 20:45:24,154 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-27 20:45:24,154 INFO sqlalchemy.engine.Engine [raw sql] {}


## List of images invalid

In [35]:
list_images_invalid = get_list_of_images_invalid()
pd.DataFrame(list_images_invalid).head(3)

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal


## BR dataset

In [36]:
query = session.query(County).distinct().all()

uf_unaccented_lower, state_unaccented_lower, county_unaccented_lower = db.get_list_uf_state_county(query)

## Define variable for build dataset

In [37]:
list_minimum_image=[5]#, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256']#, '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'

## copy images

In [38]:
level = DataTrustedIdentifier.specific_epithet_trusted
if level == DataTrustedIdentifier.genus_trusted:
    condition_level = DataTrustedIdentifier.genus_trusted.is_not(None)
else:
    condition_level = DataTrustedIdentifier.specific_epithet_trusted.is_not(None)

condition = sa.and_(DataTrustedIdentifier.country_trusted =='Brasil',
                condition_level,
                 sa.or_(uf_unaccented_lower, state_unaccented_lower))

separate_and_copy_images(condition, level, list_color, list_images_invalid, list_minimum_image, list_image_size, path_out, session)

specie: aleyreanum before: 54 after: 53
diff: ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0023115.jpeg']
matching: ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0023715.jpeg', '/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0023763.jpeg', '/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0023883.jpeg', '/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0031787.jpeg', '/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/INPA0041527.jpeg', '/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB

SystemExit: 

/home/xandao/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Close connection

In [31]:
session.close()
engine.dispose()